# Exercise 3 - SAKI

Le but de cet exercice est de trouver la facon optimale pour effectuer une série de tâches dans un entrepôt. 

Nous allons en effet utiliser le Reinforcement learning afin de trouver la meilleure voie possible pour un robot d'éffectuer un ensemble d'actions.

Pour la resolution de notre exercice nous allons utiliser le plan suivant qui nous permettra à chaque fois d'allier la théorie à la pratique.

En effet nous alons expliquer le concept du reinforcement learning et utiliser cet exercice comme un cas pratique pour illustrer et faciliter la compréhension des différentes notions.

1. Introduction : Explication du concept de reinforcement learning et des différents élements caractérisant la formalisation d'un problème de RL.

2. Elements du rl
    
    a. Agent
    
    b. Environment
    
    c. State

## Introduction

Reinforcement learning (RL) is learning what to do, how to map situations to actions, so as to maximize
a numerical reward signal. The learner is not told which actions to take, but instead must discover
which actions yield the most reward by trying them.

In short, we can say that reinforcement learning is learning by trying.

In RL there is no supervisor, only a reward signal or a real number that tells the agent how good or bad was his action. 
Feedback from the environment might be delayed over several time steps, it’s not necessarily instantaneous 
e.g. for the task of reaching a goal in a grid-world, the feedback might be at the end when the agent reaches the goal. 
The agent might spend some time exploring and wandering in the environment until it finally reaches the goal after a while to realize what were the good and bad actions it has taken.

In machine learning or supervised learning, we have a dataset that describes the environment to the algorithm and the right answers or actions to do when faced with a specific situation, and the algorithm tries to generalize from that data to new situations.

In RL the agent influences the environment through its actions which in turn affect the subsequent data it receives from the environment, it’s an active learning process.


## Elements of Reinforcement Learning


### Agent and environnement

The agent and the environment interact at each other over a sequence of discrete time steps, t = 0, 1... .

At each time step t, the agent receives some representation of the environment’s state, St ∈ S, and on that basis selects an action, At ∈ A(s).

One time step later (t + 1), in part as a consequence of its action, the agent receives a numerical reward, Rt+1 ∈ R ⊂ ℝ, and finds itself in a new state, St+1.

All the sequence of observations, actions, and rewards during the agent’s life time up to time step t is called the history,

*  Dans notre cas notre agent sera représenté par le robot qui sera chargé des rangers des objets dans notre entrepôt.

* L'environnement ici sera par contre notre entrepôt et son état sera fonction des éléments qui seront ou pas présent à l'intérieur de lui.

Beyond the agent and the environment, one can identify four main subelements of a reinforcement learning system: a policy, a reward signal, a value function, and, optionally, a model of the environment.

- __Policy : 

- __Reward signal : 

- __Value function : 

- __Model : 


### Policy

A policy defines the learning agent's way of behaving at a given time. A policy is a mapping from perceived states of the environment to actions to be taken when in those states.

The policy is the core of a reinforcement learning agent in the sense that it alone is sufficient to determine behavior. 

 It could be a deterministic policy that we want to learn from experience, or a stochastic policy.

### Reward signal

A reward signal defines the goal in a reinforcement learning problem. On each time step, the environment
sends to the reinforcement learning agent a single number called the reward. The agent’s job is to maximize the expected sum of rewards.

Since our process involves sequential decision making tasks, our actions we make early on may have a long-term consequence on our overall goal. Sometimes it may be better to sacrifice immediate reward (reward at time step Rₜ) to gain more long-term reward.

All the sources used for the realization of this article...

References :

<a href="https://towardsdatascience.com/reinforcement-learning-demystified-36c39c11ec14">Reinforcement Learning Demystified</a>

<a href="https://towardsdatascience.com/reinforcement-learning-an-introduction-to-the-concepts-applications-and-code-ced6fbfd882d">Reinforcement Learning: An Introduction to the Concepts, Applications and Code</a>

<a href="https://medium.com/coinmonks/implement-reinforcement-learning-using-markov-decision-process-tutorial-272012fdae51">Implement Reinforcement learning using Markov Decision Process</a>

<a href="http://www.cs.cmu.edu/~10601b/slides/MDP_RL.pdf">Markov Decision Process and Reinforcement Learning</a>

<a href="https://hub.packtpub.com/reinforcement-learning-mdp-markov-decision-process-tutorial/">Implement Reinforcement learning using Markov Decision Process [Tutorial]</a>

<a href="https://www.geeksforgeeks.org/markov-decision-process/">Markov Decision Process</a>

<a href="http://incompleteideas.net/book/bookdraft2017nov5.pdf">SB11 - Sutton Barto - Book - Reinforcement Learning.pdf</a>